# Laboratorio 2025
### Análisis y diseño de algoritmos distribuidos en redes
### Andrés Montoro 5.169.779-1


In [ ]:
from pydistsim.algorithm.node_algorithm import NodeAlgorithm, StatusValues
from pydistsim.algorithm.node_wrapper import NodeAccess
from pydistsim.message import Message
from pydistsim.restrictions.communication import BidirectionalLinks
from pydistsim.restrictions.reliability import TotalReliability
from pydistsim.restrictions.topological import Connectivity
from pydistsim.restrictions.knowledge import InitialDistinctValues

from pydistsim import NetworkGenerator, Simulation
from pydistsim.logging import set_log_level, LogLevels, enable_logger, logger

from pydistsim.gui import drawing as draw
%matplotlib inline
from matplotlib import pyplot as plt

import numpy as np

set_log_level(LogLevels.INFO)
enable_logger()

from utils import *

## Parte 1: 

### 1. Implementación básica del problema
- Simule un conjunto de n generales (nodos) que deben decidir si atacar o retirarse.
- Permita que hasta f generales sean bizantinos, es decir, que puedan enviar mensajes contradictorios.
- Los generales leales deberán acordar una decisión común, cumpliendo las condiciones de consistencia y validez.


### 2. Protocolos de comunicación
- Implemente el protocolo recursivo propuesto por Lamport, Shostak y Pease (OM(f)).


In [ ]:
class ByzantineGenerals(NodeAlgorithm):
    def __init__(self, simulation, *args, f=None, verbose=False, **kwargs):
        super().__init__(simulation, *args, **kwargs)
        self.f = f
        self.siege = Siege(len(self.network.nodes()) - f)
        self.messages_counter = 0
        self.logging = []

    default_params = {
        "Value" : "Value",
    }

    class Status(StatusValues):
        COMMANDER = "COMMANDER"
        GENERAL = "GENERAL"
        ATTACK = "ATTACK"
        RETREAT = "RETREAT"
        TRAITOR = "TRAITOR"
    S_init = (Status.COMMANDER, Status.GENERAL)
    S_term = (Status.ATTACK, Status.RETREAT, Status.TRAITOR)

    algorithm_restrictions = (
        BidirectionalLinks,
        Connectivity, 
        TotalReliability, # 3.A1 TODO no asumirlo?? ver notas
        InitialDistinctValues 
    )
        
    def initializer(self):
        self.apply_restrictions()
        traidores = random.sample(list(self.network.nodes()), self.f)
        commander_general = self.network.nodes_sorted()[0]
        for node in self.network.nodes():
            node.memory["tree"] = {}
            node.status = self.Status.GENERAL if node not in traidores else self.Status.TRAITOR
        if commander_general not in traidores:
            commander_general.status = self.Status.COMMANDER
        commander_general.memory["observed_success_chance"] = define_general_threshold(commander_general)
        commander_general.push_to_inbox(Message(meta_header=NodeAlgorithm.INI, destination=commander_general))
 

    def OM_commander(self, node : NodeAccess, packet : Data, sender = None):
        liutenants = node.neighbors() - {sender} if sender else node.neighbors()
        send_with_check(
            node, 
            datos=packet, 
            dest=liutenants,
            msj_type=self.default_params["Value"],
            algorithm=self,
        )


    """ 
        Voy a recibir, por cada path recursivo (u1,..., uk-1, uk) una sola llamada. Es un paso OM(m-k) 
        Caso base: mi decision es el valor que me llego. Sino:
            con nuevo path (u1,..., uk-1, uk, ui) ( me anexo al final)
            se invoca OM(m-(k+1)) enviando a n-2 liutenants (2)
            El valor recibido es mi decision en el nuevo subarbol (u1,..., uk-1, _)
        ese path me llego porque a k le llego OM(m-k+1) con (u1,...,uk-1) y me mando OM(m-k) 
        Entonces
            Lo guardo en arbol de valores que estoy esperando con key = path = (u1,...,uk-1, uk) (3)
        Si ya recibi todos los valores {v1,...,vn} placeholdeando (v1,...,vk-1, _):
            Defino con majority con ese conjunto
            Si estoy en surface layer (las tuplas tienen len 1): es la decision final
    """
    def liutenant_process(self, node: NodeAccess, datos : Data, sender = None):
        path = datos.path
        if len(path) >= self.f:
            decision = datos.value
            #TODO: que hago con esto?
        else:
            recursive_path = path + (node.memory["unique_value"],)
            node.memory["tree"][recursive_path] = datos.value
            new_datos = Data(recursive_path, datos.value)
            self.OM_commander(node, new_datos, sender)
        node.memory["tree"][path] = datos.value


    def info(self, node : NodeAccess, message : Message):
        datos : Data = message.data
        self.logging.append(f"[{f"General {node.memory['unique_value']}"}] Le llega el mensaje con data {datos}")
        self.logging.append(f"[{f"General {node.memory['unique_value']}"}] Queda con arbol {node.memory['tree']}")


    """
        IC1 exige que si el comandante es leal, todos tienen que hacerle caso
        Creo que se precisa usar al commander en llamadas sucesivas para outrulear a los traidores
        PERO no le va a llegar ningun valor de OM(m-1), porque a nadie mas llamaron con OM(m)
        Que se la juege entonces? Y espere que el resto tambien
    """
    @Status.COMMANDER
    def spontaneously(self, node: NodeAccess, _: Message):
        decision = observe(node)
        datos = Data((), decision)
        self.OM_commander(node, datos)
        if decision == GeneralDecision.ATTACK:
            self.siege.attack(node)
            node.status = self.Status.ATTACK
        else:
            self.siege.retreat(node)
            node.status = self.Status.RETREAT


    @Status.COMMANDER
    def receiving(self, node: NodeAccess, message: Message):
        datos : Data = message.data
        self.liutenant_process(node, datos, message.source)


    @Status.TRAITOR
    def spontaneously(self, node: NodeAccess, _: Message):
        datos = Data((), GeneralDecision.RETREAT)
        TraitorActions.send_confusing_signal(node, self, datos)


    @Status.TRAITOR
    def receiving(self, node: NodeAccess, message: Message):
        if self.siege.state != Siege.State.ONGOING:
            return
        #Por ahora solo envia, no le importa nada del tamaño del path ni nada
        datos : Data = message.data
        new_path =  datos.path + (node.memory["unique_value"],)
        new_datos = Data(new_path, GeneralDecision.RETREAT)
        TraitorActions.send_confusing_signal(node, self, new_datos, message.source)


    @Status.GENERAL
    def receiving(self, node: NodeAccess, message: Message):
        datos : Data = message.data
        self.liutenant_process(node, datos, message.source)
        datos : Data = message.data
        path = datos.path
        n = len(node.neighbors()) # +1; descarto al commander del que me llego OM(m) en primer lugar
        ronda = { key: value for (key, value) in node.memory["tree"].items() if len(key) == (len(path))}
        self.info(node, message)

        if len(ronda) == n:
            m = majority(ronda)
            self.logging.append(f"[{f"General {node.memory['unique_value']}"}] Decision en path {path}: {m}")
            if len(path) == 1:
                if m == GeneralDecision.ATTACK:
                    self.siege.attack(node)
                    node.status = self.Status.ATTACK
                else:
                    self.siege.retreat(node)
                    node.status = self.Status.RETREAT

    
    @Status.ATTACK
    def default(self, *args, **kwargs):    
        pass

    @Status.RETREAT
    def default(self, *args, **kwargs):    
        pass



In [ ]:
f = 2
n = 7
its = 50



net_gen = NetworkGenerator(directed=False)
net = net_gen.generate_complete_network(n)
sim = Simulation(net, check_restrictions=True)
sim.algorithms = ((ByzantineGenerals, {"f": f, "verbose" : True}),)
logger.info("----- LOGGING INFO -----")
for i in range(its):
    sim.reset()
    sim.run()
    alg = sim.algorithms[0]
    if alg.siege.state == Siege.State.FAILED:
        logger.info(f"Iteracion {i} falló")
        for node in net.nodes():
            if node.status == ByzantineGenerals.Status.TRAITOR:
                logger.info(f"{node.memory['unique_value']} era traidor")
        for log in alg.logging:
            logger.info(log)
    else:
        logger.info(f"Iteracion {i} exitosa")
    logger.info("\n\n\n")

# fig = draw.draw_current_state(sim)
# fig
# plt.close()


- Muestre como crece la complejidad en mensajes según la cantidad de fallos f. Evalúe el caso f = 1 y luego f = 2.

Si un nodo comienza una ejecución de OM(m) por ser el primer comandante, entonces será el único en ejecutar OM(m).

En caso de que un nodo ejecute OM(m) por haber recibido un mensaje OM(m+1), entonces todos los nodos demás nodos menos el comandante de OM(m+1) deben haber recibido el mensaje OM(m+1), y por lo tanto también ejecutado OM(m).

- Mida el número total de mensajes intercambiados y el tiempo necesario para llegar al consenso.


Hacer

### 3. Análisis
- Determine experimentalmente el número mínimo de nodos necesario para alcanzar consenso frente a f fallos bizantinos.
- Verifique la condición teórica n ≥ 3f + 1.

In [ ]:
def find_minimum_generals_for_consensus(f : int, its : int = None):
    net_gen = NetworkGenerator(directed=False)
    protective_max = 4*f
    n_search = f
    statistic_success = False
    while not statistic_success and n_search <= protective_max:
        n_search +=1
        statistic_success = False
        net = net_gen.generate_complete_network(n_search)
        sim = Simulation(net, check_restrictions=True)
        sim.algorithms = ((ByzantineGenerals, {"f": f, "verbose" : False}),)

        for i in range(its):
            sim.reset()
            sim.run()
            alg = sim.algorithms[0]
            statistic_success &= alg.siege.state != Siege.State.FAILED
            if not statistic_success:
                break
    return n_search if statistic_success else None

fs = np.array([2, 3, 4, 5])

for fi in fs:
    n_found = find_minimum_generals_for_consensus(fi, 25*fi)
    print(f"Para f={fi} se encontró n mínimo para alcanzar consenso n = {n_found}")
    if not n_found:
        print(f"No se encontro n minimo para f = {fi} en el espacio de busqueda permitido\n")
    elif n_found >= 3*fi + 1:
        print(f"Se cumple la condicion para f = {fi}: {n_found} ≥ 3*{fi} + 1\n")
    else:
        print(f"No se cumple la condicion para f = {fi}: {n_found} < 3*{fi} + 1\n")